In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import librosa
import sklearn

In [2]:
class Mymodel(keras.Model):
    def __init__(self):
        super(Mymodel, self).__init__()

        self.CNN = keras.Sequential([
            ## build the CNN layers
            keras.layers.Conv2D(filters=32, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=64, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=96, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Conv2D(filters=128, kernel_size=3,activation='relu',padding='same'),
            keras.layers.MaxPool2D(),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation='softmax')
        ])

    def call(self, x):
        return self.CNN(x)

In [3]:
def MFCC(fname):
    ## read the file and padding to the same 
    x, sr = librosa.load(fname)
    ## 22050* 4s
    if x.shape[0]< 88200:
        x = np.pad(x,(0, 88200-x.shape[0]),'constant')
    x = x[0:88200]
    ## mfccs
    mfccs = librosa.feature.mfcc(y=x, sr=sr,n_mfcc=40)
    ## normalize the MFCCs 
    norm_mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
    return norm_mfccs
def recong(y):
    y = y.reshape((10,))
    z = np.sort(y)
    label =['airconditioner', 'carhorn', 'childrenplaying', 'dogdark', 'drilling', 'engineidling', 'gunshot', 'jackhammer', 'siren','street_music']
    for yy in [z[-1], z[-2], z[-3]]:
        c = np.where(y==yy)
        print('Recongnized:',label[int(c[0])], 'Probability:',y[int(c[0])])

In [4]:
model = Mymodel()
model.compile(optimizer=keras.optimizers.Adam(),
            loss = keras.losses.CategoricalCrossentropy(),
             metrics=['accuracy'])
## load the pretrained model weights
# 'weights/weights' is the weights you have ever saved
model.load_weights('weights/weights')

In [5]:
fnames= ['../urbansound8K/fold10/100648-1-3-0.wav','../urbansound8K/fold9/7063-6-0-0.wav','../urbansound8K/fold8/4918-3-0-0.wav','../urbansound8K/fold7/518-4-0-0.wav']

In [6]:
for fname in fnames:
    mfccs = MFCC(fname)
    mfccs = mfccs.reshape((-1,40,173,1))
    y  = model.predict(mfccs)
    print('Recongnizing :', fname, '................')
    recong(y)

Recongnizing : ../urbansound8K/fold10/100648-1-3-0.wav ................
Recongnized: drilling Probability: 0.42905027
Recongnized: carhorn Probability: 0.32514605
Recongnized: jackhammer Probability: 0.22547002
Recongnizing : ../urbansound8K/fold9/7063-6-0-0.wav ................
Recongnized: gunshot Probability: 0.9999863
Recongnized: engineidling Probability: 1.36436465e-05
Recongnized: drilling Probability: 9.406698e-08
Recongnizing : ../urbansound8K/fold8/4918-3-0-0.wav ................
Recongnized: dogdark Probability: 0.9982309
Recongnized: drilling Probability: 0.0009892287
Recongnized: carhorn Probability: 0.00048628406
Recongnizing : ../urbansound8K/fold7/518-4-0-0.wav ................
Recongnized: drilling Probability: 0.67499924
Recongnized: airconditioner Probability: 0.17677404
Recongnized: jackhammer Probability: 0.109373316
